In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('p2_train.csv')
test = pd.read_csv('p2_test.csv')

In [3]:
train.head()

,Unnamed: 0,text,label
0,7744,longlost pictur archiv celebr glamour seneg wr...,0
1,10536,advanc understand natur world respons current ...,1
2,7043,itali test western economi bear almost total s...,1
3,5930,south africa box granni juke jab way healthier...,1
4,11660,coronaviru pandem nation tragedi hundr thousan...,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, Attention
from tensorflow.keras.callbacks import EarlyStopping

In [11]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [14]:

# LSTM model
model = Sequential()
# model.add(Embedding(input_dim=len(tagset)+1, output_dim=300, input_length=max_length))
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(64, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)
# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
43/43 [==============================] - 30s 283ms/step - loss: 0.6516 - accuracy: 0.5689
Epoch 2/15
43/43 [==============================] - 12s 268ms/step - loss: 0.5299 - accuracy: 0.7009
Epoch 3/15
43/43 [==============================] - 12s 269ms/step - loss: 0.4964 - accuracy: 0.7192
Epoch 4/15
43/43 [==============================] - 11s 258ms/step - loss: 0.3652 - accuracy: 0.8453
Epoch 5/15
43/43 [==============================] - 11s 251ms/step - loss: 0.2438 - accuracy: 0.9047
Epoch 6/15
43/43 [==============================] - 11s 250ms/step - loss: 0.1500 - accuracy: 0.9450
Epoch 7/15
43/43 [==============================] - 10s 243ms/step - loss: 0.0975 - accuracy: 0.9714
Epoch 8/15
43/43 [==============================] - 10s 235ms/step - loss: 0.0500 - accuracy: 0.9861
Epoch 9/15
43/43 [==============================] - 10s 224ms/step - loss: 0.0391 - accuracy: 0.9890
Epoch 10/15
43/43 [==============================] - 11s 248ms/step - loss: 0.0567 - accura

In [15]:
tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

11/11 [==============================] - 2s 49ms/step - loss: 0.8477 - accuracy: 0.8387


In [16]:
preds = model.predict(X_test_padded)

11/11 [==============================] - 2s 34ms/step


In [17]:
preds[0]

array([0.00101619, 0.9989838 ], dtype=float32)

In [18]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(0.0)
  else:
    preds_new.append(1.0)

In [19]:
preds_new[1]

0.0

In [20]:
y_test_array = np.array(y_test)
y_test_array[1]

1

In [21]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

F1 score =  0.8476454293628809
Precision =  0.8181818181818182
Recall =  0.8793103448275862
Accuracy =  0.8387096774193549


In [22]:
from sklearn.metrics import classification_report, roc_curve, auc, f1_score, accuracy_score, confusion_matrix
matrix = confusion_matrix(y_test, preds_new, labels = [0, 1])
mat = matrix.diagonal()/matrix.sum(axis=1)
print(classification_report(y_test, preds_new, labels = [0, 1],digits=4))
print('confusion matrix: ', mat)

              precision    recall  f1-score   support

           0     0.8636    0.7964    0.8287       167
           1     0.8182    0.8793    0.8476       174

    accuracy                         0.8387       341
   macro avg     0.8409    0.8379    0.8382       341
weighted avg     0.8404    0.8387    0.8383       341

confusion matrix:  [0.79640719 0.87931034]
